In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#hide
from RS.core2 import *
from RS.model import *

# Project name here

> Summary description here.

This file will become your README and also the index of your documentation.

## Install

`pip install your_project_name`

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
from fastai.vision.all import *

In [ ]:
df=pd.read_csv('./train.csv')

In [ ]:
df.head()

,path,valid
0,/home/staff/xin/Downloads/tianchi/train/013508.tif,0
1,/home/staff/xin/Downloads/tianchi/train/011260.tif,0
2,/home/staff/xin/Downloads/tianchi/train/009475.tif,0
3,/home/staff/xin/Downloads/tianchi/train/004362.tif,0
4,/home/staff/xin/Downloads/tianchi/train/012815.tif,0


In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(RsImage.create)),
                       TransformBlock(type_tfms=partial(RsMask.create)),
                      ),
               get_x=ColReader('path'),
               get_y=ColReader('path'),
               splitter=ColSplitter('valid'),
               item_tfms=[aug,aug2]
              )

# db.summary(source=df)

In [ ]:

dls = db.dataloaders(source=df,bs=16, num_workers=8)


In [ ]:
model=ResNestedUNet(num_classes=10,input_channels=5,deep_supervision=False)
apply_init(model)

In [ ]:
# loss=OhemCrossEntropy()

In [ ]:
def mIOU( pred,label, num_classes=10):
    pred = F.softmax(pred, dim=1)              
    pred = torch.argmax(pred, dim=1).squeeze(1)
    iou_list = list()
    present_iou_list = list()
    pred = pred.view(-1)
    label = label.view(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        if sem_class!=30:
            pred_inds = (pred == sem_class)
            target_inds = (label == sem_class)
            if target_inds.long().sum().item() == 0:
                iou_now = float('nan')
            else: 
                intersection_now = (pred_inds[target_inds]).long().sum().item()
                union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
                iou_now = float(intersection_now) / float(union_now)
                present_iou_list.append(iou_now)
            iou_list.append(iou_now)
#     present_iou_list.append(np.mean(present_iou_list))
    return np.mean(present_iou_list)

In [ ]:
class newloss(nn.Module):
    def __init__(self,):
        super().__init__()
        self.loss = CrossEntropyLossFlat(axis=1)
    def forward(self, input, target):
        return self.loss(RsImage(input),RsImage(target.long()))

In [ ]:
class OhemCrossEntropy(nn.Module): 
    def __init__(self, ignore_label=-1, thres=0.7, 
        min_kept=100000, weight=None): 
        super(OhemCrossEntropy, self).__init__() 
        self.thresh = thres
        self.min_kept = max(1, min_kept)
        self.ignore_label = ignore_label 
        self.criterion = nn.CrossEntropyLoss(weight=weight, 
                                             ignore_index=ignore_label, 
                                             reduction='none') 
    
    def forward(self, score, target, **kwargs):
        target=RsImage(target.long())
        pred = F.softmax(score, dim=1)
        pixel_losses = self.criterion(score, target).contiguous().view(-1)
        mask = target.contiguous().view(-1) != self.ignore_label         
        
        tmp_target = target.clone() 
        tmp_target[tmp_target == self.ignore_label] = 0 
        pred = pred.gather(1, tmp_target.unsqueeze(1)) 
        pred, ind = pred.contiguous().view(-1,)[mask].contiguous().sort()
        min_value = pred[min(self.min_kept, pred.numel() - 1)] 
        threshold = max(min_value, self.thresh) 
        
        pixel_losses = pixel_losses[mask][ind]
        pixel_losses = pixel_losses[pred < threshold] 
        return pixel_losses.mean()

In [ ]:
loss=OhemCrossEntropy()

In [ ]:
learn = Learner(dls,model,metrics=mIOU,loss_func=loss).to_fp16()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_one_cycle(50, 1e-3,cbs=[CSVLogger(fname='resunet5.csv',append=True),SaveModelCallback(monitor='mIOU',fname='resunet5')])

epoch,train_loss,valid_loss,mIOU,time
0,0.995570,1.031369,0.309586,01:21
1,0.999354,1.048299,0.309775,01:21
2,0.980726,1.003247,0.311629,01:21
3,0.986847,1.018933,0.311798,01:21
4,0.984862,1.003612,0.312451,01:21
5,0.970652,1.019988,0.311541,01:21
6,0.990823,1.036197,0.313517,01:21
7,0.983142,1.022808,0.310790,01:21
8,0.981431,0.996509,0.314597,01:21
9,0.963629,0.998863,0.311043,01:21


Better model found at epoch 0 with mIOU value: 0.30958580322383045.
Better model found at epoch 1 with mIOU value: 0.3097747743135482.
Better model found at epoch 2 with mIOU value: 0.31162856673624423.
Better model found at epoch 3 with mIOU value: 0.31179808363262673.
Better model found at epoch 4 with mIOU value: 0.31245149507435915.
Better model found at epoch 6 with mIOU value: 0.31351651825400817.
Better model found at epoch 8 with mIOU value: 0.3145969698426233.
Better model found at epoch 18 with mIOU value: 0.3154071372944825.
Better model found at epoch 19 with mIOU value: 0.31731310606017243.
Better model found at epoch 24 with mIOU value: 0.3176130683257932.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

